<a href="https://colab.research.google.com/github/jmhuer/Fall2021MusicResearch/blob/main/HumanFeedbackAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import re
import subprocess


def download_gdrive(id, print_stout=True):
  coomand = 'gdown https://drive.google.com/uc?id={}'.format(id)
  returned_value = subprocess.run(coomand, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  if print_stout: print(returned_value.stdout.decode("utf-8"))
  else: print("Download Complete")

In [3]:
download_gdrive("1UU9pmrtlpyElV9REbYxEHxiIdHKOmSRm", print_stout=True)

Downloading...
From: https://drive.google.com/uc?id=1UU9pmrtlpyElV9REbYxEHxiIdHKOmSRm
To: /content/responses.zip
100%|██████████| 434k/434k [00:00<00:00, 14.0MB/s]



In [4]:
!unzip responses.zip

Archive:  responses.zip
  inflating: website email.pdf       
  inflating: __MACOSX/._website email.pdf  
  inflating: website email-21.pdf    
  inflating: __MACOSX/._website email-21.pdf  
  inflating: website email-20.pdf    
  inflating: __MACOSX/._website email-20.pdf  
  inflating: website email-19.pdf    
  inflating: __MACOSX/._website email-19.pdf  
  inflating: website email-18.pdf    
  inflating: __MACOSX/._website email-18.pdf  
  inflating: website email-17.pdf    
  inflating: __MACOSX/._website email-17.pdf  
  inflating: website email-16.pdf    
  inflating: __MACOSX/._website email-16.pdf  
  inflating: website email-15.pdf    
  inflating: __MACOSX/._website email-15.pdf  
  inflating: website email-14.pdf    
  inflating: __MACOSX/._website email-14.pdf  
  inflating: website email-13.pdf    
  inflating: __MACOSX/._website email-13.pdf  
  inflating: website email-12.pdf    
  inflating: __MACOSX/._website email-12.pdf  
  inflating: website email-11.pdf    
  infl